
<html>
<head>
</head>
<body>

  <h1>Holding Period Return (HPR) Formula</h1>

  <p>The formula for Holding Period Return (HPR) is:</p>

  <p>
    <code>
      HPR = ((Ending Value - Beginning Value) + Income Received) / Beginning Value
    </code>
  </p>

  <p><strong>Where:</strong></p>

  <ul>
    <li><strong>Ending Value:</strong> The value of the investment at the end of the holding period.</li>
    <li><strong>Beginning Value:</strong> The initial value of the investment.</li>
    <li><strong>Income Received:</strong> Any income generated by the investment (e.g., dividends or interest).</li>
  </ul>

  <hr>

  <h4>Example</h4>

  <p>For example, if you buy a stock for $100, it pays a $5 dividend, and you sell it for $110, your HPR would be:</p>

  <p>
    <code>
      HPR = (($110 - $100) + $5) / $100 = $15 / $100 = 0.15 or 15%
    </code>
  </p>

  <p>This means you earned a 15% return on your initial investment over the holding period.</p>

</body>
</html>

In [1]:
# install dependencies, yahoo finance to get data

#!pip install yfinance; pandas
#!pip install plotly

In [2]:
#import dependencies

import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date, timedelta




import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings('ignore', category=UserWarning)



<h1>Part 1: Calculating HPR for a Single Stock</h1>
Define Your Investment

To calculate HPR, you need to know the initial investment details. 

Let's use Apple (AAPL) as our example. We'll simulate buying 10 shares a year ago

In [3]:
#set stock information

ticker_symbol = 'AAPL'
shares = 10
purchase_date = (date.today() - timedelta(days = 365)).strftime('%Y-%m-%d')


# fetch data
historical_data = yf.download(ticker_symbol, start = purchase_date, end = date.today())
historical_data = pd.DataFrame(historical_data)

historical_data.head()

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2024-08-08,212.317062,213.202919,207.857921,212.117996,47161100
2024-08-09,215.233429,215.770909,210.983301,211.112701,42201600
2024-08-12,216.768021,218.741081,214.844788,215.313143,38028100
2024-08-13,220.494949,221.112773,218.242856,218.242856,44155300
2024-08-14,220.943344,222.248753,218.930416,219.797379,41960600


In [4]:
try:
    purchase_price = historical_data.loc[purchase_date]['Close'].iloc[0]
    initial_invstm = purchase_price * shares
    print(f"Purchased {shares} shares for ${initial_invstm:.2f} on {purchase_date}")
except IndexError:
    print(f"Could not get information for {purchase_date}")
except ValueError:
    print(f"Not a valid number")

Purchased 10 shares for $2123.17 on 2024-08-08


**Fetch Current Data** 

Now, we get the current price and any dividends paid during the holding period.

In [5]:
if initial_invstm is not None:
    current_price = yf.download(ticker_symbol, period = '1d')['Close'].iloc[0,0]
    ending_value = current_price * shares
    
    #dividends
    dividends = yf.Ticker(ticker_symbol).history(start = purchase_date, end = date.today().strftime('%Y-%m-%d'))['Dividends'].sum()
    total_income = dividends * shares

    
    print(f"\n Current Stock Price: ${current_price:.2f} per share")
    print(f'\n Current investment value: ${ending_value:.2f}')
    print(f'\n Total Dividends Recieved: ${dividends:.2f} per share')
    print(f'\n Total income : ${total_income:.2f}')
    
          

[*********************100%***********************]  1 of 1 completed


 Current Stock Price: $226.42 per share

 Current investment value: $2264.20

 Total Dividends Recieved: $1.01 per share

 Total income : $10.10



<html>
<head>
</head>
<body>

  <h3>Holding Period Return (HPR) Formula</h3>

  <p>The formula for Holding Period Return (HPR) is:</p>

  <p>HPR = ( (Ending Value - Beginning Value) + Income ) / Beginning Value</p>

  <p><strong>Where:</strong></p>

  <ul>
    <li><strong>Ending Value:</strong> The value of the investment at the end of the holding period.</li>
    <li><strong>Beginning Value:</strong> The initial value of the investment.</li>
    <li><strong>Income:</strong> Any income generated by the investment during the holding period (e.g., dividends, interest).</li>
  </ul>

</body>
</html>

In [6]:
#calculating HPR
if initial_invstm and ending_value:
    hpr = ( (ending_value - initial_invstm) + total_income) / initial_invstm
    print(f"\nHolding Period Return (HPR) for {ticker_symbol}: {hpr:.2f}")


Holding Period Return (HPR) for AAPL: 0.07


<h1>Part 2: Calculating HPR for a Portfolio of Stocks</h1>

This part extends the single-stock logic to handle multiple assets simultaneously.


We'll use a dictionary to hold the details of our portfolio, including the ticker, number of shares, and purchase price for each stock.

In [7]:
portfolio = {
    'AAPL' : {'shares' : 10},
    'MSFT' : {'shares' : 5},
    'GOOGL' : {'shares' : 8}
}

# purchase_date = (date.today() - timedelta(days = 365)).strtime('%Y-%m-%d')

In [8]:

for key, value in portfolio.items():
    try:
        # querying for share's initial price
        stock_initial_price = yf.download(key, period = '365d' )['Close'].loc[purchase_date].iloc[0]
        portfolio[key].update({'share_initial_price' : round(stock_initial_price,2 )})

        # calculating stock's value
        stock_initial_value = (value['shares'] * value['share_initial_price'])
        portfolio[key].update({'stock_initial_value' : stock_initial_value })

        # querying for share's ending price
        stock_final_price = yf.download(key, period = '1d')['Close'].iloc[0,0]
        portfolio[key].update({'share_final_price' : round(stock_final_price, 2) })

        # calculating stock's final value
        stock_final_value = round(value['shares'] * stock_final_price , 2)
        portfolio[key].update({'stock_final_value' : round(stock_final_value, 2) })
        
        # querying for dividends
        share_div = yf.Ticker(key).history(start = purchase_date, end = date.today().strftime('%Y-%m-%d'))['Dividends'].sum()
        ttl_incm = round(share_div * value['shares'], 2)
        portfolio[key].update({'dividends' : ttl_incm })
        
    except Exception:
        print(Exception)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [9]:
df_portfolio = pd.DataFrame.from_dict(portfolio, orient = 'index')

df_portfolio.head()

,shares,share_initial_price,stock_initial_value,share_final_price,stock_final_value,dividends
AAPL,10,212.32,2123.20,226.35,2263.45,10.10
MSFT,5,399.63,1998.15,521.53,2607.65,16.20
GOOGL,8,161.25,1290.00,200.96,1607.65,6.48


In [10]:
# reorder columns and sort rows by stock_value desc

order = ['shares', 'share_initial_price', 'share_final_price', 'stock_initial_value', 'stock_final_value', 'dividends']
df_portfolio = df_portfolio[order]
df_portfolio.sort_values(by = 'stock_final_value', ascending = False, inplace = True)
df_portfolio.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, MSFT to GOOGL
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   shares               3 non-null      int64  
 1   share_initial_price  3 non-null      float64
 2   share_final_price    3 non-null      float64
 3   stock_initial_value  3 non-null      float64
 4   stock_final_value    3 non-null      float64
 5   dividends            3 non-null      float64
dtypes: float64(5), int64(1)
memory usage: 168.0+ bytes


<!DOCTYPE html>
<html>
<head>
</head>
<body>

  <h3>Again Holding Period Return (HPR) Formula</h3>

  <p>The formula for Holding Period Return (HPR) is:</p>

  <p>HPR = ( (Ending Value - Beginning Value) + Income ) / Beginning Value</p>

  <p><strong>Where:</strong></p>

  <ul>
    <li><strong>Ending Value:</strong> The value of the investment at the end of the holding period.</li>
    <li><strong>Beginning Value:</strong> The initial value of the investment.</li>
    <li><strong>Income:</strong> Any income generated by the investment during the holding period (e.g., dividends, interest).</li>
  </ul>

</body>
</html>

In [11]:
#calculating the values needed
beginning_value = df_portfolio['stock_initial_value'].sum()
ending_value = df_portfolio['stock_final_value'].sum()
div = df_portfolio['dividends'].sum()

# inputting into the formula
hpr_stock = ( (ending_value - beginning_value) + div) / beginning_value

print(f" Invested Amount: ${beginning_value} \n Current Value: ${ending_value} \n Dividends: ${div} \n \n HPR: {hpr_stock:.2f} ")

 Invested Amount: $5411.35 
 Current Value: $6478.75 
 Dividends: $32.78 
 
 HPR: 0.20 






This Notebook explores a simple way to calculate the **Holding Period Return** on Single Shares or Portfolio of stocks

<h1> Another Example Using an embedded Google Sheet </h1>

<iframe src="https://docs.google.com/spreadsheets/d/e/2PACX-1vTXkmY_6HqIEWW_Qmz6om-xDOcdP7_XhlbpRk5u-8cHMT_vP9GT1wJes6SLVGX5Z2RWHRK6057-Zv8W/pubhtml?gid=293289562&amp;single=true&amp;widget=true&amp;headers=false"></iframe>

In [16]:
from IPython.display import HTML

# Replace the URL with your Google Sheets embed link
iframe_code = """
<iframe src="https://docs.google.com/spreadsheets/d/e/2PACX-1vTXkmY_6HqIEWW_Qmz6om-xDOcdP7_XhlbpRk5u-8cHMT_vP9GT1wJes6SLVGX5Z2RWHRK6057-Zv8W/pubhtml?gid=293289562&amp;single=true&amp;widget=true&amp;headers=false" width = "100%" height = "800" frameborder = "0"></iframe>
"""

HTML(iframe_code)





  <h2>Document: HPR_Project-Daily Rates of Return-v3.csv</h2>
  <p>The document contains daily financial data with the following columns:</p>
  <ul>
    <li><strong>Date:</strong> The specific date for which the financial data is recorded.</li>
    <li><strong>Close:</strong> The closing price of an asset on the given date.</li>
    <li><strong>HPR:</strong> 'Holding Period Return', showing the daily rate of return for the asset.</li>
  </ul>

  <hr>

  <h3>Holding Period Return (HPR) Formula</h3>
  <p>The Holding Period Return (HPR) measures the total return an investor receives from an investment over a specific period. For daily data, it's calculated as:</p>
  <pre><code>HPR = (Current Close Price - Previous Close Price) / Previous Close Price</code></pre>
  <p>This formula calculates the percentage change in the closing price from one day to the next.</p>

  <hr>

  <h3>Understanding Averages in Financial Analysis</h3>
  <p>When analyzing returns, two common types of averages are used: the Arithmetic Mean and the Geometric Mean.</p>

  <h4>Arithmetic Mean (Average)</h4>
  <p>The arithmetic mean is the sum of all values divided by the number of values. It is calculated as:</p>
  <pre><code>Arithmetic Mean = &sum; Ri / n</code></pre>
  <p>Where:</p>
  <ul>
    <li><code>Ri</code> is the return for period <code>i</code></li>
    <li><code>n</code> is the number of periods</li>
  </ul>
  <p><strong>Implications for Analysis:</strong></p>
  <ul>
    <li>The arithmetic mean is useful for understanding the typical return in any single period.</li>
    <li>It is often used for forecasting future returns for a single period.</li>
    <li>However, it does not account for compounding, which is crucial when analyzing returns over multiple periods. It can overestimate the actual wealth accumulated over time.</li>
  </ul>

  <h4>Geometric Mean</h4>
  <p>The geometric mean is a more accurate measure of average return when dealing with investments that compound over multiple periods. It is calculated as:</p>
  <pre><code>Geometric Mean = ((1 + R1) * (1 + R2) * ... * (1 + Rn)) ^ (1/n) - 1</code></pre>
  <p>Where:</p>
  <ul>
    <li><code>Ri</code> is the return for period <code>i</code></li>
    <li><code>n</code> is the number of periods</li>
  </ul>
  <p><strong>Implications for Analysis:</strong></p>
  <ul>
    <li>The geometric mean represents the compound annual growth rate (CAGR) and shows the actual average rate at which an investment has grown over time.</li>
    <li>It is particularly important for financial analysis because it accounts for the effect of compounding, providing a more realistic picture of investment performance over multiple periods.</li>
    <li>It is always less than or equal to the arithmetic mean, with the difference increasing with the volatility of the returns.</li>
  </ul>

  <hr>

  <h3>Calculated Averages for Daily Rates of Return</h3>
  <p>Based on the HPR data:</p>
  <ul>
    <li><strong>Arithmetic Mean of HPR:</strong> 0.1043%</li>
    <li><strong>Geometric Mean of HPR:</strong> 0.0930%</li>
  </ul>

  <h4>Analysis of the Calculated Means:</h4>
  <ul>
    <li>The arithmetic mean (0.1043%) is slightly higher than the geometric mean (0.0930%). This difference is expected because the arithmetic mean does not account for compounding, while the geometric mean does.</li>
    <li>The arithmetic mean suggests that, on any given day, the average return of the asset is approximately 0.1043%. This might be useful for understanding the expected return over a very short period or for simple forecasting of a single day's return.</li>
    <li>The geometric mean provides a more accurate representation of the actual average daily growth rate of the investment over the entire period. It indicates that, on average, the asset has grown by approximately 0.0930% per day, considering the compounding effect of returns. This is the more appropriate measure for understanding the long-term performance and wealth accumulation of the asset.</li>
    <li>The small difference between the two means suggests that the daily volatility of returns might not be extremely high over this period. If the returns were more volatile, the gap between the arithmetic and geometric means would be larger.</li>
  </ul>